In [1]:
!pip install -q "dlt[qdrant]" "qdrant-client[fastembed]"

In [2]:
import dlt
print(dlt.__version__)

1.12.3


## Q1: 1.12.3

In [10]:
import requests
@dlt.resource(write_disposition="replace", name="zoomcamp_data")
def zoomcamp_data():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            yield doc

In [11]:
from dlt.destinations import qdrant

qdrant_destination = qdrant(
  qd_path="db.qdrant", 
)

In [12]:
pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data"

)
load_info = pipeline.run(zoomcamp_data())
print(pipeline.last_trace)

Run started at 2025-07-10 16:07:25.128740+00:00 and COMPLETED in 5.40 seconds with 4 steps.
Step extract COMPLETED in 0.93 seconds.

Load package 1752163645.6261098 is EXTRACTED and NOT YET LOADED to the destination and contains no failed jobs

Step normalize COMPLETED in 0.05 seconds.
Normalized data for the following tables:
- zoomcamp_data: 948 row(s)
- _dlt_pipeline_state: 1 row(s)

Load package 1752163645.6261098 is NORMALIZED and NOT YET LOADED to the destination and contains no failed jobs

Step load COMPLETED in 3.94 seconds.
Pipeline zoomcamp_pipeline load step completed in 3.92 seconds
1 load package(s) were loaded to destination qdrant and into dataset zoomcamp_tagged_data
The qdrant destination used /workspaces/llm-zoomcamp/cohorts/2025/workshops/dlt/db.qdrant location to store data
Load package 1752163645.6261098 is LOADED and contains no failed jobs

Step run COMPLETED in 5.40 seconds.
Pipeline zoomcamp_pipeline load step completed in 3.92 seconds
1 load package(s) were l

## Q2: 948

In [19]:
import json

import os

current_dir = os.getcwd()

file_path = current_dir +"/db.qdrant/meta.json"
with open(file_path, "r") as f:
    data = json.load(f)

In [22]:
print(json.dumps(data,indent=2))


{
  "collections": {
    "zoomcamp_tagged_data": {
      "vectors": {
        "fast-bge-small-en": {
          "size": 384,
          "distance": "Cosine",
          "hnsw_config": null,
          "quantization_config": null,
          "on_disk": null,
          "datatype": null,
          "multivector_config": null
        }
      },
      "shard_number": null,
      "sharding_method": null,
      "replication_factor": null,
      "write_consistency_factor": null,
      "on_disk_payload": null,
      "hnsw_config": null,
      "wal_config": null,
      "optimizers_config": null,
      "init_from": null,
      "quantization_config": null,
      "sparse_vectors": null,
      "strict_mode_config": null
    },
    "zoomcamp_tagged_data__dlt_pipeline_state": {
      "vectors": {
        "fast-bge-small-en": {
          "size": 384,
          "distance": "Cosine",
          "hnsw_config": null,
          "quantization_config": null,
          "on_disk": null,
          "datatype": null,
   

In [24]:
data["collections"]["zoomcamp_tagged_data"]["vectors"]

{'fast-bge-small-en': {'size': 384,
  'distance': 'Cosine',
  'hnsw_config': None,
  'quantization_config': None,
  'on_disk': None,
  'datatype': None,
  'multivector_config': None}}

## Q3: fast-bge-small-en